<h1><center>Sentiment Analysis Classifier with BERT</center></h1>

This project is based on Manning's book "Transfer Learning for NLP" (chapter 2).
The goal here is:

1. Curate a dataset with reviews from imdb classic dataset
2. Create a Pandas dataframe from iy
3. Create a simple bag-of-words model from the above content. Simple because it is based on term frequency (tf) only.
4. Choose one baseline classifier from Logistic Regression and Gradient Boosting Machine
5. Accuracy is the metric of choice as the dataset is balanced and consists of two classes
6. Train a SentimentAnalysis classifier based on BERT embeddings

But before starting let's make sure we have the correct libraries versions installed, namelly tensorflow and bert-tensorflow.

In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

     |████████████████████████████████| 110.5 MB 29 kB/s 
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


Now we will import required Python libraries and the dataset. To download the dataset I will use the bash script get_aclImdb.sh. It downloads and extracts the compressed archive into ./data/aclImdb. It requires execution privilege (sudo chmod +x get_aclImdb.sh).

In [2]:
import pandas as pd 
import numpy as np 
import pickle as pck 
import os.path
from os import path

# download dataset
!./get_aclImdb.sh

aclImdb already downloaded.


# Hyperparameters

In [43]:
max_tokens = 256 # maximum number of tokens per review
max_chars = 20 # maximum size of a token.

Three helper methods to tokenize, remoce stopwords, remove puntuation and convert to lowercase

In [21]:
def tokenize(text):

    if text==None or text=='' or type(text)=='list':
        tokens = ""
    else:
        tokens = text.split(' ')[: max_tokens]

    return tokens

In [39]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

def stopword_removal(tokens):

    filtered_tokens = [token for token in tokens if (token not in stopwords and token is not None)]
    filtered_tokens = filtered_tokens[:max_tokens]
        
    return filtered_tokens


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/baosiek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
import re

def clean_tokens(tokens):

    cleaned_tokens = []
    for token in tokens:
        token = token.lower()
        token = re.sub(r'[\W\d]', "", token)[:max_chars]
        cleaned_tokens.append(token)

    return cleaned_tokens

In [24]:
def shuffle_data(reviews_np, sentiment_list):

    shuffle_index = np.random.permutation(len(sentiment_list))
    reviews_np = reviews_np[shuffle_index]
    sentiments_np = np.asarray(sentiment_list)[shuffle_index]

    return reviews_np, sentiments_np

In [31]:
def load_dataset(path):

    reviews, sentiments = [], []
    for folder, sentiment in (('neg', 0), ('pos', 1)):
        folder = os.path.join(path, folder)
        for name in os.listdir(folder):
            with open(os.path.join(folder, name), 'r') as reader:
                text = reader.read()

            text = tokenize(text)
            text = clean_tokens(text)
            text = stopword_removal(text)

            reviews.append(text)
            sentiments.append(sentiment)

    reviews_np = np.array(reviews)
    reviews_np, sentiments_np = shuffle_data(reviews_np, sentiments)

    return reviews_np, sentiments_np

In [45]:
train_path = os.path.join('data/aclImdb', 'train')
raw_data, raw_label = load_dataset(train_path)

In [46]:
print(raw_data[41])

['muslim', 'women', 'western', 'africa', 'married', 'life', 'hands', 'abusive', 'husbands', 'hard', '', 'community', 'may', 'explicitly', 'endorse', 'behaviour', 'equally', 'may', 'yet', 'ready', 'see', 'criminal', 'attitude', 'course', 'enables', 'continue', 'fortunately', 'letter', 'cameroonian', 'law', 'promises', 'equality', 'documentary', 'follows', 'real', 'life', 'exploits', 'various', 'female', 'practitioners', 'cameroonian', 'legal', 'system', 'attempt', 'secure', 'justice', 'number', 'women', 'children', 'notable', 'apart', 'uplifting', 'central', 'story', 'spite', 'informality', 'courts', 'actually', 'pragmatically', 'progressive', 'case', 'actually', 'bought', 'program', 'also', 'gives', 'fascinating', 'insight', 'whole', 'cameroonian', 'lifestyle', 'aside', 'awful', 'crimes', 'committed', 'featured', 'cases', 'seems', 'amazingly', 'emotional', 'joyous', 'compared', 'enjoyed', 'inhabitants', 'europe', 'north', 'america', 'concede', 'comment', 'may', 'betray', 'naiveté', 'pa